Model Zoo -- Simple RNN for
Sentiment Classification<br>
a binary classification problem with two labels, positive and negative

In [0]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random

torch.backends.cudnn.deterministic=True

In [0]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

vocabulary_size = 20000
learning_rate = 1e-4
batch_size = 128
num_epochs = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_dim = 128
hidden_dim = 256
output_dim = 1

In [3]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(RANDOM_SEED), split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 29.5MB/s]


Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [5]:
TEXT.build_vocab(train_data, max_size = vocabulary_size)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


the TEXT.vocab dictionary will contain the word counts and indices. The reason why the number of words is vocabulary_size + 2 is that it contains two special tokens for padding and unknown words: $<unk>$ and $<pad>$

In [0]:
# create dataset iterators
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = batch_size,
    device = device)

In [10]:
# testing the iterators (note that the number of rows depends on the longest document in the respective batch)
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text.size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([1196, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([60, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


In [0]:
# the model part
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        # [sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        # [sentence len, batch size, embedding size] =>
        # output: [sentence len, batch size, hidden size]
        # hidden: [1, batch size, hidden size]
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0)).view(-1)

In [0]:
input_dim = len(TEXT.vocab)
embedding_dim = 64
hidden_dim = 128
output_dim = 1

torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [0]:
# training

In [0]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            logits = model(batch_data.text)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float() / num_examples * 100

In [15]:
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        ### FORWARD AND BACK PROP
        logits = model(batch_data.text)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{num_epochs:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, device):.2f}%')

Epoch: 001/015 | Batch 000/157 | Cost: 0.7097
Epoch: 001/015 | Batch 050/157 | Cost: 0.6934
Epoch: 001/015 | Batch 100/157 | Cost: 0.7013
Epoch: 001/015 | Batch 150/157 | Cost: 0.6949
training accuracy: 49.96%
valid accuracy: 49.82%
Time elapsed: 7.25 min
Epoch: 002/015 | Batch 000/157 | Cost: 0.6912
Epoch: 002/015 | Batch 050/157 | Cost: 0.6967
Epoch: 002/015 | Batch 100/157 | Cost: 0.6917
Epoch: 002/015 | Batch 150/157 | Cost: 0.6928
training accuracy: 50.02%
valid accuracy: 49.22%
Time elapsed: 14.64 min
Epoch: 003/015 | Batch 000/157 | Cost: 0.6925
Epoch: 003/015 | Batch 050/157 | Cost: 0.6927
Epoch: 003/015 | Batch 100/157 | Cost: 0.7023
Epoch: 003/015 | Batch 150/157 | Cost: 0.6938
training accuracy: 50.01%
valid accuracy: 50.46%
Time elapsed: 22.03 min
Epoch: 004/015 | Batch 000/157 | Cost: 0.6931
Epoch: 004/015 | Batch 050/157 | Cost: 0.6928
Epoch: 004/015 | Batch 100/157 | Cost: 0.6912
Epoch: 004/015 | Batch 150/157 | Cost: 0.6954
training accuracy: 50.10%
valid accuracy: 49.8

In [0]:
# save the model we trained
torch.save(model, 'model.pkl')

In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
print('probability positive:')
predict_sentiment(model, "I really love this movie, this movie is so great!")